In [1]:
#Check Python Version
import sys
sys.version

'3.8.2 (default, Jan 31 2023, 18:34:03) \n[GCC 12.2.0]'

In [2]:
#Upgrade pip
%pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [3]:
#Check TF & TFX Versioning
import tensorflow as tf
print(tf.__version__)
from tfx import v1 as tfx
print(tfx.__version__)


2023-02-06 19:55:01.801418: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-06 19:55:01.969473: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-06 19:55:01.969526: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-06 19:55:03.316559: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

2.11.0
1.12.0


In [1]:
#Setup Variables as importer_node_playground
import os

# Pipeline name
PIPELINE_NAME = "importer_node_playground"

# Output directory to store artifacts generated from the pipeline.
PIPELINE_ROOT = './artifacts'
# Path to a SQLite DB file to use as an MLMD storage.
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')
# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

# Folder path to data
DATA_ROOT = './data/'

from absl import logging
logging.set_verbosity(logging.INFO)  # Set default logging level.

In [3]:
# Create list of tuples with key value pair: {split_name, gs://path/to/directory}
split_uris = [
    ('Split_train', './tfrecords/1/Split-train/data_tfrecord-00000-of-00001.gz'),
    ('Split_eval', './tfrecords/1/Split-eval/data_tfrecord-00000-of-00001.gz')
    ]

In [25]:
#Test ExampleGen Artifact Output
def _create_pipeline(
  pipeline_name: str,
  pipeline_root: str,
  data_root: str,
  metadata_path: str
  ) -> tfx.dsl.Pipeline:

  # Brings data into the pipeline.
  example_gen = tfx.components.CsvExampleGen(input_base=data_root)

  # # Importer Node
  # importer_test = tfx.dsl.Importer(
  #   source_uri='./tfrecords/examples/1/Split-eval/data_tfrecord-00000-of-00001.gz',
  #   artifact_type=tfx.types.standard_artifacts.Schema).with_id(
  #         'importer_test')

  # Computes statistics over data for visualization and schema generation.
  # statistics_gen = tfx.components.StatisticsGen(
  #   examples=example_gen.outputs['examples'])

  components = [
    example_gen,
    # importer_test,
    # statistics_gen
  ]

  return tfx.dsl.Pipeline(
    pipeline_name=PIPELINE_NAME,
    pipeline_root=PIPELINE_ROOT,
    metadata_connection_config=tfx.orchestration.metadata
      .sqlite_metadata_connection_config(metadata_path),
    components=components
)

In [26]:
tfx.orchestration.LocalDagRunner().run(
  _create_pipeline(
      pipeline_name=PIPELINE_NAME,
      pipeline_root=PIPELINE_ROOT,
      data_root=DATA_ROOT,
      metadata_path=METADATA_PATH)
  )

INFO:absl:Using deployment config:
 executor_specs {
  key: "CsvExampleGen"
  value {
    beam_executable_spec {
      python_executor_spec {
        class_path: "tfx.components.example_gen.csv_example_gen.executor.Executor"
      }
    }
  }
}
custom_driver_specs {
  key: "CsvExampleGen"
  value {
    python_class_executable_spec {
      class_path: "tfx.components.example_gen.driver.FileBasedDriver"
    }
  }
}
metadata_connection_config {
  database_connection_config {
    sqlite {
      filename_uri: "metadata/importer_node_playground/metadata.db"
      connection_mode: READWRITE_OPENCREATE
    }
  }
}

INFO:absl:Using connection config:
 sqlite {
  filename_uri: "metadata/importer_node_playground/metadata.db"
  connection_mode: READWRITE_OPENCREATE
}

INFO:absl:Component CsvExampleGen is running.
INFO:absl:Running launcher for node_info {
  type {
    name: "tfx.components.example_gen.csv_example_gen.component.CsvExampleGen"
  }
  id: "CsvExampleGen"
}
contexts {
  contexts {
    

jo


INFO:absl:Examples generated.
INFO:absl:Value type <class 'NoneType'> of key version in exec_properties is not supported, going to drop it
INFO:absl:Value type <class 'list'> of key _beam_pipeline_args in exec_properties is not supported, going to drop it
INFO:absl:Cleaning up stateless execution info.
INFO:absl:Execution 6 succeeded.
INFO:absl:Cleaning up stateful execution info.
INFO:absl:Publishing output artifacts defaultdict(<class 'list'>, {'examples': [Artifact(artifact: uri: "./artifacts/CsvExampleGen/examples/6"
custom_properties {
  key: "input_fingerprint"
  value {
    string_value: "split:single_split,num_files:1,total_bytes:25648,xor_checksum:1675458490,sum_checksum:1675458490"
  }
}
custom_properties {
  key: "span"
  value {
    int_value: 0
  }
}
, artifact_type: name: "Examples"
properties {
  key: "span"
  value: INT
}
properties {
  key: "split_names"
  value: STRING
}
properties {
  key: "version"
  value: INT
}
base_type: DATASET
)]}) for execution 6
INFO:absl:Met

In [9]:
# [SPLIT_TRAIN] View Dataset Artifact as tf.record
train_uri = os.path.join('./artifacts/CsvExampleGen/examples/1/', 'Split-train')

tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

for tfrecord in dataset:
  # Prints out tf.record
  print(tfrecord)
  
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()

  # Prints out parsed tfrecord as JSON
  example.ParseFromString(serialized_example)
  print(example)


tf.Tensor(b'\n\x85\x01\n\x1d\n\x11flipper_length_mm\x12\x08\x12\x06\n\x04\x114\x1c>\n\x10\n\x07species\x12\x05\x1a\x03\n\x01\x00\n\x17\n\x0bbody_mass_g\x12\x08\x12\x06\n\x04UU\x95>\n\x1c\n\x10culmen_length_mm\x12\x08\x12\x06\n\x04\xc8S\x82>\n\x1b\n\x0fculmen_depth_mm\x12\x08\x12\x06\n\x04\xab\xaa*?', shape=(), dtype=string)
features {
  feature {
    key: "body_mass_g"
    value {
      float_list {
        value: 0.2916666567325592
      }
    }
  }
  feature {
    key: "culmen_depth_mm"
    value {
      float_list {
        value: 0.6666666865348816
      }
    }
  }
  feature {
    key: "culmen_length_mm"
    value {
      float_list {
        value: 0.2545454502105713
      }
    }
  }
  feature {
    key: "flipper_length_mm"
    value {
      float_list {
        value: 0.1525423675775528
      }
    }
  }
  feature {
    key: "species"
    value {
      int64_list {
        value: 0
      }
    }
  }
}

tf.Tensor(b'\n\x85\x01\n\x1b\n\x0fculmen_depth_mm\x12\x08\x12\x06\n\x041\x0c

2023-02-06 19:55:48.668864: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-06 19:55:48.668941: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-06 19:55:48.668981: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (alexanderho1998.c.googlers.com): /proc/driver/nvidia/version does not exist
2023-02-06 19:55:48.669556: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
# [SPLIT_EVAL] View Dataset Artifact
eval_uri = os.path.join('./artifacts/CsvExampleGen/examples/1/', 'Split-eval')

tfrecord_filenames = [os.path.join(eval_uri, name)
                      for name in os.listdir(eval_uri)]

dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

for tfrecord in dataset.take(3):
  # Prints out tf.record
  print(tfrecord)

  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  
  # Prints out parsed tfrecord as JSON
  example.ParseFromString(serialized_example)
  print(example)

tf.Tensor(b'\n\x85\x01\n\x1b\n\x0fculmen_depth_mm\x12\x08\x12\x06\n\x04\xf4<\x0f?\n\x17\n\x0bbody_mass_g\x12\x08\x12\x06\n\x049\x8e\x83>\n\x1d\n\x11flipper_length_mm\x12\x08\x12\x06\n\x04\x114\x1c>\n\x10\n\x07species\x12\x05\x1a\x03\n\x01\x00\n\x1c\n\x10culmen_length_mm\x12\x08\x12\x06\n\x04\x105}>', shape=(), dtype=string)
features {
  feature {
    key: "body_mass_g"
    value {
      float_list {
        value: 0.2569444477558136
      }
    }
  }
  feature {
    key: "culmen_depth_mm"
    value {
      float_list {
        value: 0.5595238208770752
      }
    }
  }
  feature {
    key: "culmen_length_mm"
    value {
      float_list {
        value: 0.24727272987365723
      }
    }
  }
  feature {
    key: "flipper_length_mm"
    value {
      float_list {
        value: 0.1525423675775528
      }
    }
  }
  feature {
    key: "species"
    value {
      int64_list {
        value: 0
      }
    }
  }
}

tf.Tensor(b'\n\x85\x01\n\x17\n\x0bbody_mass_g\x12\x08\x12\x06\n\x04\xc7q\x0